In [18]:
import joblib
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# Install xlrd package
%pip install xlrd

all_df = pd.read_excel('TestDatasetExample.xls', index_col=False)
all_df.drop('ID', axis=1, inplace=True)
all_df.head()

saved_model = joblib.load('pcr_classification_model.joblib')
model = saved_model['model']
scaler = saved_model['scaler']
lda = saved_model['lda']
selected_features_indices = saved_model['selected_features_indices']

Note: you may need to restart the kernel to use updated packages.


### Pre-Process testing data

In [19]:
from sklearn.impute import SimpleImputer

# Replace missing values with median of the column
imputer = SimpleImputer(strategy="median", missing_values=999)
all_df[:] = imputer.fit_transform(all_df)

# Standardize the data
Xs = scaler.transform(all_df)

/var/folders/xq/rx4p81p162zf4pn_29d95ftc0000gn/T/ipykernel_93691/515753628.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.  0.5 1. ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  all_df[:] = imputer.fit_transform(all_df)


### Apply Feature Selection and Dimensionality Reduction training strategy

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Select required features (ER, HER2 and Gene)
non_mri_features = Xs[:, selected_features_indices]

# Select MRI features
mri_indices = list(range(11, Xs.shape[1]))
mri = Xs[:, mri_indices]

# Apply LDA to MRI features
Xs_lda = lda.transform(Xs[:, 11:])

# Combine required features with LDA transformed features
Xs = np.hstack((non_mri_features, Xs_lda))

### Predict Classification

In [21]:
model.predict(Xs)

array([0, 1, 0])